In [99]:
import csv, json
import geojson
from geojson import Feature, FeatureCollection, Point, dump
import pandas as pd


In [100]:
ufo = pd.read_csv(r'UFO Files/us_ufo_csv.csv')
ufo.tail()

,Unnamed: 0,datetime,city,state,country,shape,duration (seconds),latitude,longitude
65109,80327,9/9/2013 21:15,nashville,tn,us,light,600.0,36.165833,-86.784444
65110,80328,9/9/2013 22:00,boise,id,us,circle,1200.0,43.613611,-116.202500
65111,80329,9/9/2013 22:00,napa,ca,us,other,1200.0,38.297222,-122.284444
65112,80330,9/9/2013 22:20,vienna,va,us,circle,5.0,38.901111,-77.265556
65113,80331,9/9/2013 23:00,edmond,ok,us,cigar,1020.0,35.652778,-97.477778


In [101]:
# create the function
def ufo_to_geojson(ufo, properties, latitude='latitude', longitude='longitude'):
    """
    Turn a dataframe containing point data into a geojson formatted python dictionary

    df : the dataframe to convert to geojson
    properties : a list of columns in the dataframe to turn into geojson feature properties
    lat : the name of the column in the dataframe that contains latitude data
    lng : the name of the column in the dataframe that contains longitude data
    """

    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    # x is the equivalent of the row, df.iterrows converts the dataframe in to a pd.series object
    # the x is a counter and has no influence
    for x, row in ufo.iterrows():

        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [float(row.longitude),float(row.latitude)]

        # be aware that the dataframe is a pd.series
        #print('rowitem converts to ndarray(numpy) :\n ', row)
        # convert the array to a pandas.serie
        geo_props = pd.Series(row)

        # for each column, get the value and add it as a new feature property
        # prop determines the list from the properties
        for prop in properties:

            #loop over the items to convert to string elements

            #convert to string
            if type(geo_props[prop]) == float:
                #print('ok')
                geo_props[prop] = str(int(geo_props[prop]))

            # now create a json format, here we have to make the dict properties
            feature['properties'][prop] = geo_props[prop]

        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
        
    return geojson

with open('ufomap.geojson','w') as f:
    json.dump(ufo_to_geojson(ufo,['latitude','longitude']),f)

with open('ufomap2.geojson','w') as f:
    json.dump(ufo_to_geojson(ufo,['datetime','latitude','longitude']),f)